In [2]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [972 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,060 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,324 kB]
Hit:13 http://ppa.launchpad.net/graphics-dri

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()
     


In [4]:

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()
     

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [5]:
df.count()

145431

In [6]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with total_votes equal or greater than 20
total_votes_20plus_df = df.filter(col("total_votes") >= 20)
total_votes_20plus_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12569493| RV4UH0OWW97K4|B0149HT55K|     947161997|Megaman Legacy Co...|Digital_Video_Games|          5|           19|         22|   N|                N|Mega Man Legacy C...|What is Mega Man?...| 2015-08-29|
|         US|   40702997| RG6GT6G9WNYT1|B013XJ2M8M|     628348017|Rocket League [On...|Digital_Video_Games| 

In [7]:
total_votes_20plus_df.count()

3342

In [8]:
# Filter the data for helpful_votes/total_votes is >= 50%
helpful_votes_50plus_df = total_votes_20plus_df.filter(col("helpful_votes")/col("total_votes") >= .50)
helpful_votes_50plus_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12569493| RV4UH0OWW97K4|B0149HT55K|     947161997|Megaman Legacy Co...|Digital_Video_Games|          5|           19|         22|   N|                N|Mega Man Legacy C...|What is Mega Man?...| 2015-08-29|
|         US|   40702997| RG6GT6G9WNYT1|B013XJ2M8M|     628348017|Rocket League [On...|Digital_Video_Games| 

In [9]:
helpful_votes_50plus_df.count()

1685

In [10]:
# Filter for data where a review was written as part of the Vine program
paid_vine_df = helpful_votes_50plus_df.filter(col("vine") == 'Y')
paid_vine_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [11]:
paid_vine_df.count()

0

In [12]:
# Filter for data where a review was not part of the Vine program
unpaid_vine_df = helpful_votes_50plus_df.filter(col("vine") == 'N')
unpaid_vine_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12569493| RV4UH0OWW97K4|B0149HT55K|     947161997|Megaman Legacy Co...|Digital_Video_Games|          5|           19|         22|   N|                N|Mega Man Legacy C...|What is Mega Man?...| 2015-08-29|
|         US|   40702997| RG6GT6G9WNYT1|B013XJ2M8M|     628348017|Rocket League [On...|Digital_Video_Games| 

In [13]:
# Finding total # of review paid
paid_vine_df.count()
     

0

In [14]:
# Finding total # of 5-star reviews paid
paid_star_rating_df = paid_vine_df.filter(col("star_rating") == 5)
paid_star_rating_df.count()

0

In [19]:
# Finding percentage of 5-star reviews paid
# percent_paid = (paid_star_rating_df.count()/paid_vine_df.count())*100
# print("Percentage of 5-star reviews for paid Vine program was: %f" % percent_paid + "%")

0

0

In [16]:
# Finding total # of review unpaid
unpaid_vine_df.count()
     

1685

In [17]:
# Finding total # of 5-star reviews unpaid
unpaid_star_rating_df = unpaid_vine_df.filter(col("star_rating") == 5)
unpaid_star_rating_df.count()

631

In [18]:
# Finding percentage of 5-star reviews unpaid
percent_unpaid = (unpaid_star_rating_df.count()/unpaid_vine_df.count())*100
print("Percentage of 5-star reviews for unpaid Vine program was: %f" % percent_unpaid + "%")

Percentage of 5-star reviews for unpaid Vine program was: 37.448071%
